# Dump Model to Excel

## Libraries

In [1]:
import os
import datetime
import traceback
import logging
import sqlite3
import sys

from logging.handlers import TimedRotatingFileHandler
from dotenv import load_dotenv
from sqlalchemy import event
from sqlalchemy import create_engine

import pandas as pd

## Environment Setting

In [2]:
use_sqlite = False
fhandler = TimedRotatingFileHandler("logs/dump_model.log", when="midnight", interval=1)
fhandler.suffix = "%Y%m%d"
logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[fhandler, logging.StreamHandler(sys.stdout)],
    )
logger = logging.getLogger(__name__)

database_folder = "database"
database_name = f"{database_folder}/main.db"

load_dotenv()

excel_file = "excelfiles/{0}.csv"

## Connecting to Database

In [3]:
my_conn = None
if use_sqlite:
  my_conn = sqlite3.connect(database_name)
else:
  my_conn = create_engine(os.getenv("PG_DATA_CONN"))  

## Routine

In [4]:
SQL_TABLES = ""
if use_sqlite:
  SQL_TABLES = "select * from sqlite_schema"
else:
  SQL_TABLES = "select table_name as name from information_schema.tables WHERE table_shema = 'main' AND table_type = 'BASE TABLE'"

tables = pd.read_sql(, my_conn)

for idx in range(tables['name'].size):
  logger.info(f"Reading table {tables['name'][idx]}...")
  table_data = pd.read_sql(f"select * from {tables['name'][idx]}", my_conn)
  if table_data.size > 0:
    logger.info(f"Saving table {tables['name'][idx]} to file...")
    table_data.to_csv(excel_file.format(tables['name'][idx]))


2022-09-13 13:30:03,449 - INFO - Reading table fact_programs...
2022-09-13 13:30:04,824 - INFO - Saving table fact_programs to file...
